# Reconstruction using idwt algorithm

![](./idwt.png)

- cA - approximation coefficient
- cD - detail coefficient
- highpass - using rec_hi filter bank
- lowpass - using rec_lo filter bank
- upsampling is done using dyadic upsampling algorithm - inserting zeros in even-indexed elements
- thanks to orthogonal properties of the `sym4` wavelet we can reconstruct the signal without losses

In [276]:
import matplotlib as mpl
import numpy as np
import pywt
import seaborn as sns

sns.set_theme()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

Dyadic upsampling. Add zeroes between (by default) even indexed values. 

In [277]:
def dyadup(a, even=True):
    start = 1 if even else 0
    out = np.zeros(len(a) * 2, dtype=a.dtype)
    out[start::2] = a
    return out

Example:

In [278]:
print(dyadup(np.array([1,2,3,4,5])))

[0 1 0 2 0 3 0 4 0 5]


## Inverse discrete wavelet transform

In [279]:
def convolve_1d(s, kernel):
    rev_kernel = kernel[::-1].copy()
    padsize = kernel.size-1
    s = np.pad(s, padsize, mode='constant')
    n_steps = s.size - kernel.size + 1

    result = np.zeros(n_steps, dtype=np.double)
    n_ker = kernel.size
    for i in range(n_steps):
        result[i] = np.dot(s[i: i + n_ker], rev_kernel)
    return result

In [280]:
def idwt(ca: np.array, cd: np.array, rec_high: np.array, rec_low: np.array) -> np.array:
    if len(ca) != len(cd):
        ca = ca[:-1] # cut the last value from approximate coefficients if length is different
    _ca = dyadup(ca, even=True) # dyadic upsampling
    _cd = dyadup(cd, even=True)
    halfa = convolve_1d(_ca, np.array(rec_low)) # compute next approximate coefficients
    halfb = convolve_1d(_cd, np.array(rec_high)) # compute next detail coefficients

    sig = halfa + halfb
    
    # `wkeep` in the diagram above. Remove unnecessary values.
    _padlen = len(rec_low)
    lo = _padlen - 1
    hi = len(sig) - (_padlen - 2)
    return sig[lo:hi]

## Signal reconstruction

Next approximate coefficient is an output of previous `idwt()`. Until required level is reached. Level can be deduced from the length of the coefficient list. 

In [281]:
def reconstruct(coeffs: list, rec_high: np.array, rec_low: np.array) -> np.array:
    res = idwt(coeffs[0], coeffs[1], rec_high=rec_high, rec_low=rec_low)
    level = len(coeffs)
    for i in range(2, level):
        res = idwt(res, coeffs[i], rec_high=rec_high, rec_low=rec_low)
    return res

## Test against pyWavelets `waverec()`

In [282]:
wavelet = pywt.Wavelet('sym4')
dec_lo, dec_hi, rec_lo, rec_hi = wavelet.filter_bank

sg = np.arange(1, 71)
coef = pywt.wavedec(sg, wavelet=wavelet, level=4)

print(reconstruct(coef, rec_high=rec_hi, rec_low=rec_lo))
print(pywt.waverec(coef, wavelet=wavelet))

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70.]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70.]
